<a href="https://colab.research.google.com/github/Cairo-Henrique/BQC-Quantum-Tech/blob/main/Bloco_1/BQC_Aula_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Brazil Quantum Camp

**Aula 5**: Introdução à Programação II

In [ ]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from functools import reduce

In [ ]:
ket_0 = np.array([1, 0], dtype=complex)
ket_1 = np.array([0, 1], dtype=complex)

## Funções e Portas Quânticas Apresentadas em Aula

In [ ]:
def get_op_n_qubits(gate_matrix, target_qubit, total_qubits):
    """Constrói a matriz de operação para N qubits"""
    # 1. Cria lista de Identidades: [I, I, I, ...]
    gates = [np.eye(2, dtype=complex)] * total_qubits

    # 2. Substitui a Identidade no
    # alvo pela porta desejada
    gates[target_qubit] = gate_matrix

    # 3. Faz o kron de tudo: I kron I kron U kron I ...
    # reduce aplica kron acumulativamente na lista
    full_op = reduce(np.kron, gates)

    return full_op


def get_cnot_matrix(n_qubits, control, target):
    """Constrói a matriz CNOT para N qubits item a item."""
    size = 2**n_qubits
    control = n_qubits - 1 - control  # Ajusta índice
    target = n_qubits - 1 - target  # Ajusta índice
    U = np.zeros((size, size), dtype=complex)
    # Itera sobre todos os estados da base (colunas)
    for col in range(size):
        # Verifica se o bit de controle está ativo
        if (col >> control) & 1:
            # Se sim, o destino (linha) é o índice com o bit alvo invertido
            row = col ^ (1 << target)
        else:
            # Se não, o estado não muda
            row = col
        # Define a transição |row><col|
        U[row, col] = 1.0
    return U


def plot(state: np.ndarray, **kwargs):
    """Plota o estado quântico usando Plotly Express."""
    # 1. Preparando os dados
    # flatten(): Transforma matriz (4,1) em array plano (4,)
    raio = np.abs(state.flatten())
    angulo = np.angle(state.flatten())
    # 2. Gerando labels (|00>, |01>...) dinamicamente
    n_qubits = int(np.log2(len(state)))
    estados = [f"|{i:0{n_qubits}b}>" for i in range(len(raio))]
    # 3. Plotando
    # x=Estado, y=Amplitude, color=Fase
    fig = px.bar(x=estados, y=raio, color=angulo, **kwargs)
    return fig


def apply_mcz(state, qubits):
    """Aplica Z (fase -1) se TODOS os qubits listados forem 1."""
    n = int(np.log2(len(state)))
    # 1. Cria uma máscara com os qubits envolvidos
    mask = 0
    for q in qubits:
        mask |= 1 << (n - 1 - q)
    # 2. Itera sobre os estados da base
    for i in range(len(state)):
        # Verifica se todos os bits da máscara estão ligados em i
        if (i & mask) == mask:
            state[i] *= -1
    return state

# Usando a igualdade X = HZH, podemos facilmente criar uma MCX a partir de uma MCZ:
# Basta aplicar H, Z condicional e H no alvo
def apply_mcx(state, controls, target):
    """Aplica a porta MCX (C...CX) a um estado quântico."""
    # 1. Aplica H no alvo
    state = apply_gate(state, "H", target)

    # 2. Aplica MCZ
    state = apply_mcz(state, controls, target)

    # 3. Aplica H no alvo novamente
    state = apply_gate(state, "H", target)

    return state


def get_projectors(target, n_qubits):
    """Constrói os operadores M0 e M1 expandidos para N qubits."""
    # Lista de Identidades
    ops0, ops1 = [np.eye(2, dtype=complex)] * n_qubits, [
        np.eye(2, dtype=complex)
    ] * n_qubits
    # Substitui a identidade no alvo pelo projetor
    ops0[target] = np.array([[1, 0], [0, 0]], dtype=complex)  # |0><0|
    ops1[target] = np.array([[0, 0], [0, 1]], dtype=complex)  # |1><1|
    # Cria as matrizes grandes (Tensor Product)
    M0, M1 = reduce(np.kron, ops0), reduce(np.kron, ops1)
    return M0, M1


def get_probability(state, M):
    """Calcula P(m) = || M|psi> ||^2"""
    # Projeta o estado (sem normalizar)
    projected_state = M @ state

    # A norma ao quadrado é a probabilidade
    prob = np.linalg.norm(projected_state) ** 2

    # Retorna o estado projetado
    # para reuso e a probabilidade
    return prob, projected_state


def normalize(projected_state, prob):
    """Normaliza o estado: |psi'> = M|psi> / sqrt(P)"""

    return projected_state / np.sqrt(prob)


def measure(state, target_qubit):
    n = int(np.log2(len(state)))
    # 1. Obter Operadores
    M0, M1 = get_projectors(target_qubit, n)
    # 2. Calcular Probabilidades e Projeções
    prob0, state0 = get_probability(state, M0)
    prob1, state1 = get_probability(state, M1)
    # 3. Sorteio (Natureza escolhe o resultado)
    # Nota: np.random.choice precisa de soma exata 1.0
    outcome = np.random.choice([0, 1], p=[prob0, 1 - prob0])
    # 4. Colapso Irreversível
    final_state = normalize(state0, prob0) if outcome == 0 else normalize(state1, prob1)
    return outcome, final_state


def measure_all(state):
    """Mede todos os qubits de um estado quântico."""
    n = int(np.log2(len(state)))
    results = []
    for qubit in range(n):
        bit, state = measure(state, qubit)  # Atualiza o estado a cada passo!
        results.append(bit)
    return results, state  # Estado final será |b0 b1 ... bn>


def bloch_coordinates(ket):
    """Calcula as coordenadas de Bloch de um qubit."""
    bra = np.conj(ket.T)
    # Valor Esperado <O> = <psi|O|psi>
    x = (bra @ GATES["X"] @ ket).item().real
    y = (bra @ GATES["Y"] @ ket).item().real
    z = (bra @ GATES["Z"] @ ket).item().real
    return go.Scatter3d(
        x=[x],
        y=[y],
        z=[z],
        mode="markers",
        marker={"size": 10, "color": "red"},
    )


def bloch():
    """Gera a superfície da esfera."""
    phi = np.linspace(0, np.pi, 100)
    theta = np.linspace(0, 2 * np.pi, 200)
    phi, theta = np.meshgrid(phi, theta)

    # Conversão Esférica -> Cartesiana
    x = np.sin(phi) * np.cos(theta)
    y = np.sin(phi) * np.sin(theta)
    z = np.cos(phi)

    return go.Surface(x=x, y=y, z=z, showscale=False, opacity=0.5)


def bloch_plot(ket):
    """Plota o estado de um qubit na esfera de Bloch."""
    # 1. Cria a Esfera
    sphere = bloch()
    # 2. Calcula o Ponto do Qubit
    point = bloch_coordinates(ket)
    # 3. Retorna a Figura Combinada
    return go.Figure(data=[sphere, point])


def create_state(n_qubits: int) -> np.ndarray:
    """Cria um estado quântico inicializado no estado
    |0...0> para um dado número de qubits."""
    qubits = n_qubits * [ket_0]
    estado_composto = reduce(np.kron, qubits)
    return estado_composto


def apply_gate(state: np.ndarray, gate_name: str, target: int, parametro=None) -> np.ndarray:
    """Aplica uma porta quântica de um qubit a um estado quântico."""
    n_qubits = int(np.log2(len(state)))
    gate = GATES[gate_name]
    if parametro is not None:
        gate = gate(parametro)
    operator = get_op_n_qubits(gate, target, n_qubits)
    return operator @ state


def apply_cnot(state: np.ndarray, control: int, target: int) -> np.ndarray:
    """Aplica a porta CNOT a um estado quântico."""
    n_qubits = int(np.log2(len(state)))
    operator = get_cnot_matrix(n_qubits, control, target)
    return operator @ state


GATES = {
    "X": np.array([[0, 1], [1, 0]], dtype=complex),
    "Y": np.array([[0, -1j], [1j, 0]], dtype=complex),
    "Z": np.array([[1, 0], [0, -1]], dtype=complex),
    "H": (1 / np.sqrt(2)) * np.array([[1, 1], [1, -1]], dtype=complex),
    "RX": lambda theta: np.array(
        [
            [np.cos(theta / 2), -1j * np.sin(theta / 2)],
            [-1j * np.sin(theta / 2), np.cos(theta / 2)],
        ],
        dtype=complex,
    ),
    "RY": lambda theta: np.array(
        [
            [np.cos(theta / 2), -np.sin(theta / 2)],
            [np.sin(theta / 2), np.cos(theta / 2)],
        ],
        dtype=complex,
    ),
    "RZ": lambda theta: np.array(
        [
            [np.exp(-1j * theta / 2), 0],
            [0, np.exp(1j * theta / 2)],
        ],
        dtype=complex,
    ),
    "P": lambda theta: np.array(
        [
            [1, 0],
            [0, np.exp(1j * theta)],
        ],
        dtype=complex,
    ),
}

## **Exemplo**: Medida e Emaranhamento

**Estado Separável $\left|++\right>$**

$\frac{1}{2}(\left|00\right>+\left|01\right>+\left|10\right>+\left|11\right>)$

Medir o Qubit 0 **não afeta** a probabilidade do Qubit 1.
Eles são independentes.



In [ ]:
psi = np.array([1, 0, 0, 0], dtype=complex)
psi = np.kron(GATES["H"], GATES["H"]) @ psi

plot(psi, title="Estado após aplicar H em ambos os qubits").show()

result0, psi = measure(psi, 0)

print("Resultado da medição do qubit 0:", result0)

plot(psi, title="Estado após medição do qubit 0").show()

result1, psi = measure(psi, 1)

print("Resultado da medição do qubit 1:", result1)

plot(psi, title="Estado após medição do qubit 1").show()

Resultado da medição do qubit 0: 0


Resultado da medição do qubit 1: 0



**Estado de Bell**

$\frac{1}{\sqrt{2}}(\left|++\right>+\left|00\right>)$

1.  Medir Q0. Probabilidade 50/50.
2.  Suponha resultado $0$.
3.  O estado colapsa para $\left|01\right>$.
4.  A medida de Q1 agora será $0$
    com **100%** de certeza.

O colapso de um qubit emaranhados afeta os demais.

In [ ]:
bell00 = np.array([1, 0, 0, 0], dtype=complex)
bell00 = np.kron(GATES["H"], np.eye(2)) @ bell00
bell00 = get_cnot_matrix(2, control=0, target=1) @ bell00

plot(bell00, title="Estado após aplicar CNOT").show()

result0, bell00 = measure(bell00, 0)
print("Resultado da medição do qubit 0:", result0)

plot(bell00, title="Estado após medição do qubit 0").show()

result1, bell00 = measure(bell00, 1)
print("Resultado da medição do qubit 1:", result1)

plot(bell00, title="Estado após medição do qubit 1").show()

Resultado da medição do qubit 0: 1


Resultado da medição do qubit 1: 1


## Esfera de Bloch


In [ ]:
psi = np.array([1, 0], dtype=complex)
psi = GATES["RX"](np.pi / 4) @ psi
psi = GATES["RZ"](np.pi / 5) @ psi

bloch_plot(psi).show()

## 🛠️ **Desafio**: A Classe `QuantumCircuit`

Nossa meta é preencher essa estrutura.
O estado quântico (`self.state`) deve ser persistente entre as chamadas.

In [ ]:
class QuantumCircuit:
    def __init__(self, n_qubits: int):
        """Inicializa o circuito com |00...0>."""
        qubits = n_qubits * [ket_0]
        estado_composto = reduce(np.kron, qubits)
        self.state = estado_composto
        self.n_qubits = n_qubits

    def x(self, target: int):
        """Aplica porta Pauli-X (NOT)."""
        self.state = apply_gate(self.state, "X", target)
        return self.state

    def y(self, target: int):
        """Aplica porta Pauli-Y."""
        self.state = apply_gate(self.state, "Y", target)
        return self.state

    def z(self, target: int):
        """Aplica porta Pauli-Z."""
        self.state = apply_gate(self.state, "Z", target)
        return self.state

    def h(self, target: int):
        """Aplica porta Hadamard."""
        self.state = apply_gate(self.state, "H", target)
        return self.state

    def rx(self, theta: float, target: int):
        """Aplica rotação Rx por um ângulo theta."""
        self.state = apply_gate(self.state, "RX", target, parametro=theta)
        return self.state

    def ry(self, theta: float, target: int):
        """Aplica rotação Ry por um ângulo theta."""
        self.state = apply_gate(self.state, "RY", target, parametro=theta)
        return self.state

    def rz(self, theta: float, target: int):
        """Aplica rotação Rz por um ângulo theta."""
        self.state = apply_gate(self.state, "RZ", target, parametro=theta)
        return self.state

    def p(self, phi: float, target: int):
        """Aplica porta de Fase (Phase Shift)."""
        self.state = apply_gate(self.state, "P", target)
        return self.state

    def cnot(self, control: int, target: int):
        """Aplica CNOT (CX)."""
        self.state = apply_cnot(self.state, control, target)
        return self.state

    def mcx(self, controls: list[int], target: int):
        """Aplica Toffoli Generalizado (Multi-Controlled X)."""
        self.state = apply_mcx(self.state, controls, target)
        return self.state

    def mcz(self, qubits: list[int]):
        """Aplica Z Controlado (fase -1 se todos forem 1)."""
        self.state = apply_mcz(self.state, qubits)
        return self.state

    def measure(self, qubits: list[int]) -> list[int]:
        """Mede os qubit, colapsa o estado e retorna os bit."""
        results = []
        for qubit in qubits:
            bit, self.state = measure(self.state, qubit)
            results.append(bit)
        return results

    def histogram(self, **kwargs) -> go.Figure:
        """
        Retorna um gráfico de barras com as probabilidades
        e fase de cada estado.
        """
        # 1. Preparando os dados
        state = self.state
        # flatten(): Transforma matriz (4,1) em array plano (4,)
        raio = np.abs(state.flatten())
        angulo = np.angle(state.flatten())
        # 2. Gerando labels (|00>, |01>...) dinamicamente
        n_qubits = int(np.log2(len(state)))
        estados = [f"|{i:0{n_qubits}b}>" for i in range(len(raio))]
        # 3. Plotando
        # x=Estado, y=Amplitude, color=Fase
        fig = px.bar(x=estados, y=raio, color=angulo, **kwargs)
        return fig

## 🧪 **Exercício**: Codificação Superdensa

Usando a classe `QuantumCircuit`, implemente
o protocolo de Codificação Superdensa.

* Alice envia 2 bits clássicos transferindo 1 qubit.
* Usa portas $I, X, Z, ZX$ dependendo da mensagem.

![](https://upload.wikimedia.org/wikipedia/commons/b/b7/Superdense_coding.png)

In [ ]:
# Implementação da codificação superdensa

# Bits a serem enviados de Alice para Bob
b1 = 0
b2 = 1

qc = QuantumCircuit(2)

# Prepare and share a Bell pair |phi+>
qc.h(0)
qc.cnot(0, 1)

# Sender encodes bits
if b2 == 1:
    qc.x(0)
if b1 == 1:
    qc.z(0)

# Receiver decodes bits
qc.cnot(0, 1)
qc.h(0)
measure_b1, measure_b2 = qc.measure([0,1])

print(f'Alice enviou {b1}{b2} \nBob mediu {measure_b1}{measure_b2}')

qc.histogram(title="Estado Inicial").show()

Alice enviou 01 
Bob mediu 01



## 🧪 **Exercício**: Teletransporte Quântico

Usando a classe `QuantumCircuit`, implemente
o protocolo de Teletransporte Quântico.


* Alice transfere o
  estado $\left|\psi\right>$ para Bob.
* Usando um par de Bell
  $$\left|\Phi\right>^+ = \frac{1}{\sqrt{2}}(\left|01\right>+\left|10\right>)$$



![](https://upload.wikimedia.org/wikipedia/commons/d/dc/Quantum_teleportation_circuit.svg)

Teste com $\left|\psi\right> = \frac{1}{\sqrt{2}}(\left|0\right>-\left|1\right>)$



In [ ]:
# Implementação do Teleporte Quântico

qc = QuantumCircuit(3)

# Criar |psi>
qc.x(0)
qc.h(0)

# Criar |phi+>
qc.h(1)
qc.cnot(1, 2)

# Transferir estado quasi-psi para Bob
qc.cnot(0, 1)
qc.h(0)
m_0, m_1 = qc.measure([0, 1])

# Bob transforma e obtém o estado psi
if m_0 == 1:
    qc.z(2)
if m_1 == 1:
    qc.x(2)

#m_2 = qc.measure([2])
qc.histogram(title="Estado Final").show()